In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from datetime import datetime, timedelta

In [30]:
# function to add the seconds of the row to the timestamp of the row 

def addSecondsToTimestamp(timestamp, secondsToAdd):
    return timestamp + timedelta(seconds=secondsToAdd)


def formDataset(path):
    convert_to_timestamp = lambda x: datetime.fromtimestamp(int(x))

    dataset = pd.read_csv(path, parse_dates=['SubmitTime'], date_parser=convert_to_timestamp)

    dataset['StopTime'] = dataset[['SubmitTime', 'RunTime']].apply(lambda x: addSecondsToTimestamp(*x), axis=1)


    dataset.drop(dataset.select_dtypes(['object']), inplace=True, axis=1)
    dataset = dataset[dataset["RunTime"] > 0]
    dataset = dataset[dataset["NProc"] > 0]
    dataset = dataset[dataset["UsedCPUTime"] > 0]
    dataset = dataset[dataset["UsedMemory"] > 0]

    # keep only needed columns
    dataset = dataset[['SubmitTime', 'RunTime', 'NProc', 'UsedCPUTime', 'UsedMemory', 'StopTime']]
    try:
        cancelled_jobs = dataset[dataset["ReqTime"] < dataset["RunTime"]]

        #drop columns with 'ReqTime' == -1
        cancelled_jobs.drop(cancelled_jobs["ReqTime"] == -1, inplace=True)
        cancelled_jobs.to_csv(f"./datasets/dropped/dropped-{path.split('/')[-1]}", index=False)
    except:
        pass



    dataset.to_csv( f"./datasets/formed/formed-{path.split('/')[-1]}", index=False)

In [31]:
for csv in glob.glob(pathname="./datasets/raw/*.csv"):
    print(csv)
    if csv in ['./datasets/raw/fastStorage.csv', './datasets/raw/rnd.csv']:
        continue
    formDataset(csv)

./datasets/raw/grid5000.csv
./datasets/raw/das2.csv


/var/folders/rz/x38ry1m901jcjct47ydtrgph0000gn/T/ipykernel_40359/3365228901.py:10: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(path, parse_dates=['SubmitTime'], date_parser=convert_to_timestamp)


./datasets/raw/rnd.csv
./datasets/raw/nordugrid.csv
./datasets/raw/auvergrid.csv
./datasets/raw/fastStorage.csv
./datasets/raw/sharcnet.csv
